In [ ]:
!pip -q install python-docx sentence-transformers faiss-cpu numpy pandas scikit-learn tqdm

In [ ]:
from docx import Document
import re
import random

#read doc
DATA_PATH = "./Guilan-Food.docx"
doc = Document(DATA_PATH)

lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]

foods = []
cur = None

for line in lines:
    m = re.match(r"^\d+\.\s*(.+)$", line)
    if m:
        if cur:
            foods.append(cur)
        cur = {"title": m.group(1).strip(), "content": ""}
    elif cur:
        cur["content"] += line + "\n"

if cur:
    foods.append(cur)

print("تعداد غذاها:", len(foods))
print("نمونه اولین غذا:")
print("Title:", foods[0]["title"])
print(foods[0]["content"][:150], "...\n")



def split_sections(content: str):
    ingredients, instructions, desc = "", "", ""
    section = "desc"

    for line in content.split("\n"):
        if not line.strip():
            continue
        if line.startswith("مواد لازم"):
            section = "ingredients"
            continue
        if line.startswith("طرز تهیه"):
            section = "instructions"
            continue

        if section == "desc":
            desc += line + "\n"
        elif section == "ingredients":
            ingredients += line + "\n"
        elif section == "instructions":
            instructions += line + "\n"

    return {
        "desc": desc.strip(),
        "ingredients": ingredients.strip(),
        "instructions": instructions.strip()
    }

for f in foods:
    parts = split_sections(f["content"])
    f.update(parts)

print("بعد از split:")
print("Title:", foods[0]["title"])
print("Desc:", foods[0]["desc"][:60])
print("Ingredients:", foods[0]["ingredients"].split("\n")[:3])
print("Instructions:", foods[0]["instructions"][:60], "\n")


TEMPLATES_ING = [
    "مواد لازم {title} چیست؟",
    "برای درست کردن {title} چه موادی نیاز است؟",
    "{title} با چه چیزهایی درست می‌شود؟"
]

TEMPLATES_INS = [
    "طرز تهیه {title} چطور است؟",
    "روش پخت {title} چگونه است؟",
    "چطور می‌توان {title} را درست کرد؟"
]

TEMPLATES_DESC = [
    "{title} چیست؟",
    "درباره {title} توضیح بده",
    "{title} چه نوع غذایی است؟"
]

#dataset QA
pairs = []
for idx, food in enumerate(foods):
    if food["ingredients"]:
        for q in TEMPLATES_ING:
            pairs.append({
                "query": q.format(title=food["title"]),
                "doc_id": idx,
                "answer": food["ingredients"],
                "section": "ingredients"
            })
    if food["instructions"]:
        for q in TEMPLATES_INS:
            pairs.append({
                "query": q.format(title=food["title"]),
                "doc_id": idx,
                "answer": food["instructions"],
                "section": "instructions"
            })
    if food["desc"]:
        for q in TEMPLATES_DESC:
            pairs.append({
                "query": q.format(title=food["title"]),
                "doc_id": idx,
                "answer": food["desc"],
                "section": "desc"
            })

print("تعداد کل پرسش–پاسخ:", len(pairs))
print("نمونه:")
for p in random.sample(pairs, 5):
    print("Q:", p["query"])
    print("Answer (شروع):", p["answer"][:60], "...\n")


تعداد غذاها: 9
نمونه اولین غذا:
Title: رشته خشکار
رشته خشکار یکی از دسرهای محبوب و خوشمزه در گیلان است که طعمی شیرین و مطبوع دارد. این دسر معمولاً در مراسمات خاص و یا به عنوان یک میان‌وعده دلپذیر سرو  ...

بعد از split:
Title: رشته خشکار
Desc: رشته خشکار یکی از دسرهای محبوب و خوشمزه در گیلان است که طعمی
Ingredients: ['آرد برنج:   1 پیمانه', 'شیر:  2 پیمانه', 'شکر:  1 پیمانه']
Instructions: برای تهیه رشته خشکار ابتدا آرد برنج را در یک تابه با کمی کره 

تعداد کل پرسش–پاسخ: 81
نمونه:
Q: روش پخت کولی غورابیج یا کوله غورابه چگونه است؟
Answer (شروع): این خورشت تا حدی به خورشت فسنجان شبیه است و برای تهیه آن به  ...

Q: برای درست کردن خورشت شش انداز چه موادی نیاز است؟
Answer (شروع): بادمجان قلمی بدون تخم : 5 عدد
پیاز سرخ کرده :4 قاشق سوپخوری
 ...

Q: گمج کباب با چه چیزهایی درست می‌شود؟
Answer (شروع): گوشت بدون چربی و قورمه ای خرد شده :250 گرم
پیاز نگینی شده :ی ...

Q: خورشت شش انداز با چه چیزهایی درست می‌شود؟
Answer (شروع): بادمجان قلمی بدون تخم : 5 عدد
پیاز سرخ کرده :4 قاشق سوپخوری
 ...

Q: م

In [ ]:
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from tqdm import tqdm


corpus = []
corpus_map = []
for idx, f in enumerate(foods):
    if f["ingredients"]:
        corpus.append(f["ingredients"])
        corpus_map.append((idx, "ingredients"))
    if f["instructions"]:
        corpus.append(f["instructions"])
        corpus_map.append((idx, "instructions"))
    if f["desc"]:
        corpus.append(f["desc"])
        corpus_map.append((idx, "desc"))

print("تعداد کل داکیومنت‌ها:", len(corpus))


model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")
#embedding
corpus_emb = model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

dim = corpus_emb.shape[1]
index = faiss.IndexFlatIP(dim)
index.add(corpus_emb)



def evaluate(model, pairs, index, corpus_emb, corpus_map, k_list=(1,3,5,10)):
    rr, sims = [], []
    hits = {k: 0 for k in k_list}

    for ex in tqdm(pairs, desc="Evaluating"):
        q_emb = model.encode([ex["query"]], convert_to_numpy=True, normalize_embeddings=True)
        D, I = index.search(q_emb, max(k_list)) #most simiular indexes to query --> I

        rank = None #rank with D
        for r, doc_id in enumerate(I[0], start=1):
            if corpus_map[doc_id][0] == ex["doc_id"] and corpus_map[doc_id][1] == ex["section"]:
                rank = r
                break

        rr.append(1/rank if rank else 0.0)
        for k in k_list:
            if rank and rank <= k:
                hits[k] += 1

        top1_id = I[0][0]
        ans_emb = model.encode([ex["answer"]], convert_to_numpy=True, normalize_embeddings=True)
        cos = float((ans_emb @ corpus_emb[top1_id])[0])
        sims.append(cos)

    results = {
        "MRR": float(np.mean(rr)),
        "Hit@k": {k: hits[k] / len(pairs) for k in k_list},
        "CosineSimilarity(avg)": float(np.mean(sims)) #check cosine sim between real ans and model ans
    }
    return results

metrics = evaluate(model, pairs, index, corpus_emb, corpus_map)
print("Baseline metrics:")
print(metrics)


تعداد کل داکیومنت‌ها: 27


Evaluating: 100%|██████████| 81/81 [00:01<00:00, 80.65it/s]

Baseline metrics:
{'MRR': 0.2955173427395649, 'Hit@k': {1: 0.1728395061728395, 3: 0.32098765432098764, 5: 0.4691358024691358, 10: 0.6790123456790124}, 'CosineSimilarity(avg)': 0.5450877372497394}


In [ ]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
#dataset
train_examples = []
for ex in pairs:
    train_examples.append(InputExample(texts=[ex["query"], ex["answer"]]))

import random
random.shuffle(train_examples)
val_size = int(0.15 * len(train_examples))
val_data = train_examples[:val_size]
train_data = train_examples[val_size:]

train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

ft_model = SentenceTransformer("sentence-transformers/distiluse-base-multilingual-cased-v2")

loss = losses.MultipleNegativesRankingLoss(ft_model)

num_epochs = 3
warmup_steps = int(len(train_loader) * num_epochs * 0.1)

ft_model.fit(
    train_objectives=[(train_loader, loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    show_progress_bar=True,
    use_amp=True
)

SAVE_PATH = "./ft-distiluse-guilan"
ft_model.save(SAVE_PATH)
print("مدل فاین‌تیون ذخیره شد:", SAVE_PATH)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss


مدل فاین‌تیون ذخیره شد: ./ft-distiluse-guilan


In [ ]:
ft_corpus_emb = ft_model.encode(corpus, convert_to_numpy=True, normalize_embeddings=True)

index_ft = faiss.IndexFlatIP(ft_corpus_emb.shape[1])
index_ft.add(ft_corpus_emb)

ft_metrics = evaluate(ft_model, pairs, index_ft, ft_corpus_emb, corpus_map)

print("Baseline:", metrics)
print("Fine-Tuned:", ft_metrics)


Evaluating: 100%|██████████| 81/81 [00:01<00:00, 66.25it/s]

Baseline: {'MRR': 0.2955173427395649, 'Hit@k': {1: 0.1728395061728395, 3: 0.32098765432098764, 5: 0.4691358024691358, 10: 0.6790123456790124}, 'CosineSimilarity(avg)': 0.5450877372497394}
Fine-Tuned: {'MRR': 0.9094650205761315, 'Hit@k': {1: 0.8271604938271605, 3: 1.0, 5: 1.0, 10: 1.0}, 'CosineSimilarity(avg)': 0.9000939399371912}


In [ ]:
import gradio as gr

def answer_question(query, top_k=3):
    q_emb = ft_model.encode([query], convert_to_numpy=True, normalize_embeddings=True)
    D, I = index_ft.search(q_emb, top_k)

    results = []
    for rank, idx in enumerate(I[0], start=1):
        food_id, section = corpus_map[idx]
        title = foods[food_id]["title"]
        text = corpus[idx][:400] + "..."
        results.append(f"{rank}. {title} [{section}]\n{text}\n")

    return "\n\n".join(results)

demo = gr.Interface(
    fn=answer_question,
    inputs=[
        gr.Textbox(label="سوال خود را اینجا وارد کنید"),
        gr.Slider(1, 5, value=3, step=1, label="Top-K نتایج")
    ],
    outputs=gr.Textbox(label="پاسخ/نتایج"),
    title="سیستم پرسش‌وپاسخ درباره غذاهای گیلان",
    description="یک سؤال بپرسید (مثلا: مواد لازم میرزا قاسمی چیست؟ یا طرز تهیه باقلا قاتوق چطور است؟)"
)

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://86476f51565f70b93c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
